In [35]:
import pandas as pd
from FederatedDataset.PartitionTypes.underrepresented_partition import (
    UnderrepresentedPartition,
)
import torch
from collections import Counter


In [36]:
unfair_dataset = pd.read_csv("../data/celeba/unfair_train.csv")

In [37]:
# unfair_dataset["Smiling"] = [
#     1 if item == 1 else 0 for item in unfair_dataset["Smiling"]
# ]
# unfair_dataset["Male"] = [1 if item == 1 else 0 for item in unfair_dataset["Male"]]

In [38]:
unfair_dataset.head()

,Unnamed: 0.1,Unnamed: 0,image_id,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,...,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young,split,Target
0,0,0,000001.jpg,-1,1,1,-1,-1,-1,-1,...,1,-1,1,-1,1,-1,-1,1,0.137514,1
1,1,1,000002.jpg,-1,-1,-1,1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,1,-0.432273,1
2,2,2,000003.jpg,-1,-1,-1,-1,-1,-1,1,...,-1,1,-1,-1,-1,-1,-1,1,1.841473,0
3,3,3,000004.jpg,-1,-1,1,-1,-1,-1,-1,...,1,-1,1,-1,1,1,-1,1,-0.080797,1
4,4,4,000005.jpg,-1,1,1,-1,-1,-1,1,...,-1,-1,-1,-1,1,-1,-1,1,1.045482,1


In [39]:
unfair_dataset.shape

(145381, 45)

In [51]:
df_train = pd.read_csv("/mnt/NAS/user/luca.corbucci/data/celeba/train_original.csv")
df_test = pd.read_csv("/mnt/NAS/user/luca.corbucci/data/celeba/test_original.csv")

# reduce the size of the unfair_dataset by random sampling
reduced_unfair_train = df_train.sample(frac=0.05, random_state=42)
reduced_unfair_test = df_test.sample(frac=0.05, random_state=42)

In [52]:
reduced_unfair_train["Smiling"].value_counts()

-1    4222
 1    3899
Name: Smiling, dtype: int64

In [53]:
reduced_unfair_train["Male"].value_counts()

-1    4729
 1    3392
Name: Male, dtype: int64

In [54]:
reduced_unfair_train.shape

(8121, 44)

In [55]:
reduced_unfair_train.to_csv("../data/celeba/unfair_train_reduced.csv")
reduced_unfair_test.to_csv("../data/celeba/unfair_test_reduced.csv")

: 

In [4]:
unfair_dataset["Smiling"].value_counts()


0    84071
1    61310
Name: Smiling, dtype: int64

In [6]:
unfair_dataset["Male"].value_counts()


0    94816
1    50565
Name: Male, dtype: int64

In [7]:
combinations = []
for index, row in unfair_dataset.iterrows():
    label = row["Smiling"]
    sensitive_attribute = row["Male"]
    combinations.append((label, sensitive_attribute))
print(Counter(combinations))

Counter({(1, 0): 51310, (0, 0): 43506, (0, 1): 40565, (1, 1): 10000})


: 

In [8]:
labels = unfair_dataset["Smiling"].values
labels


array([1, 1, 0, ..., 1, 1, 0])

In [9]:
sensitive_attribute = list(unfair_dataset["Male"].values)


In [12]:
pool_size = 100
alpha = 5
ratio = 0.3

partitions_index_list = UnderrepresentedPartition.do_partitioning(
    labels=torch.tensor(labels),
    sensitive_features=torch.tensor(sensitive_attribute),
    num_partitions=pool_size,
    total_num_classes=2,
    alpha=alpha,
    ratio=ratio,
)


In [21]:
labels_and_sensitive = []
lenght = 0


for index_list in partitions_index_list:
    tmp = []
    tmp_labels = []
    tmp_sens = []
    counter_1 = 0
    for index in index_list:
        tmp.append((labels[index], sensitive_attribute[index]))
        tmp_labels.append(labels[index])
        tmp_sens.append(sensitive_attribute[index])
        if labels[index] == 1 and sensitive_attribute[index] == 1:
            counter_1 += 1
    print(len(tmp))
    lenght += len(tmp)
    labels_and_sensitive.append(Counter(tmp))
    print(
        Counter(tmp),
        " ||||| LABELS: ",
        Counter(tmp_labels),
        "|||| SENSITIVE VALUES: ",
        Counter(tmp_sens),
        "Male, Smiling: ",
        counter_1,
    )
    print("\n")

2735
Counter({(0, 1): 2393, (1, 1): 342})  ||||| LABELS:  Counter({0: 2393, 1: 342}) |||| SENSITIVE VALUES:  Counter({1: 2735}) Male, Smiling:  342


1767
Counter({(0, 1): 1381, (1, 1): 386})  ||||| LABELS:  Counter({0: 1381, 1: 386}) |||| SENSITIVE VALUES:  Counter({1: 1767}) Male, Smiling:  386


2223
Counter({(0, 1): 1925, (1, 1): 298})  ||||| LABELS:  Counter({0: 1925, 1: 298}) |||| SENSITIVE VALUES:  Counter({1: 2223}) Male, Smiling:  298


892
Counter({(0, 0): 658, (1, 0): 234})  ||||| LABELS:  Counter({0: 658, 1: 234}) |||| SENSITIVE VALUES:  Counter({0: 892}) Male, Smiling:  0


987
Counter({(0, 0): 604, (1, 0): 383})  ||||| LABELS:  Counter({0: 604, 1: 383}) |||| SENSITIVE VALUES:  Counter({0: 987}) Male, Smiling:  0


1091
Counter({(0, 0): 567, (1, 0): 524})  ||||| LABELS:  Counter({0: 567, 1: 524}) |||| SENSITIVE VALUES:  Counter({0: 1091}) Male, Smiling:  0


1310
Counter({(1, 0): 846, (0, 0): 464})  ||||| LABELS:  Counter({1: 846, 0: 464}) |||| SENSITIVE VALUES:  Counter({

: 

In [ ]:
print("Totale: ", lenght)